## Names: 

#### Mohamed Afif Chifaoui

#### Ricardo Vazquez Alvarez

---

# Linear and Discrete Models First Homework


## Problem statement:

A service company operates in three shifts (Morning, Afternoon, Night) and needs to determine the number of employees assigned to each shift. The objective is to minimize labor costs while ensuring sufficient staffing levels for service demand.

Firstly, we can determine that there are two types of sets for the model (the sets of shifts and the employee types). 

1. $S$: Set of Shifts (Morning, Afternoon, Nights)
2. $E$: Set of employee types (Regular, Part-time, Temporary, On-call, Vacation, Training).

The parameters to be used in the objective function and the constraints will be the following:

1. $C_{e}$: Cost per hour for each employee type. ($e \in E$)
2. $D_{s}$: Service demand for each shift. ($s \in S$).
3. $M_{s}$: Maximum overtime hours allowed for each shift. ($s \in S$)
4. $U_{e}$: Maximum allowed number for each employee type. ($e \in E$, Part-time, Temporary, On-call, Vacation, Training).

The variables are those employees with their type ($e$) to be assigned to a shift ($s$).
- $X_{es}$: Number of employees of type $e$ assigned to shift $s$.

Leading to the Objective function:

$$\text{Minimize} \;\; Z = \sum_{e\in E}^{}\sum_{s\in S}^{} C_{e}\cdot X_{es}$$



### Decision Variables:

The following are the individual decision variables that will be used:

- $x_{1}$: Number of employees assigned to the Morning shift.
- $x_{2}$: Number of employees assigned to the Afternoon shift.
- $x_{3}$: Number of employees assigned to the Night shift.
- $x_{4}$: Overtime hours for the Morning shift.
- $x_{5}$: Overtime hours for the Afternoon shift.
- $x_{6}$: Overtime hours for the Night shift.
- $x_{7}$: Number of employees assigned to the Morning shift.
- $x_{8}$: Number of employees assigned to the Afternoon shift.
- $x_{9}$: Number of employees assigned to the Night shift.
- $x_{10}$: Number of temporary employees assigned to the Morning shift.
- $x_{11}$: Number of temporary employees assigned to the Afternoon shift.
- $x_{12}$: Number of temporary employees assigned to the Night shift.
- $x_{13}$: Number of on-call employees for the Morning shift.
- $x_{14}$: Number of on-call employees for the Afternoon shift.
- $x_{15}$: Number of on-call employees for the Night shift.
- $x_{16}$: Number of employees on vacation during the Morning shift.
- $x_{17}$: Number of employees on vacation during the Afternoon shift.
- $x_{18}$: Number of employees on vacation during the Night shift.
- $x_{19}$: Number of employees on training during the Morning shift.
- $x_{20}$: Number of employees on training during the Afternoon shift.
- $x_{21}$: Number of employees on training during the Night shift.


### Constraints:

1. Shift Coverage Constraints:
$$\sum_{e\in E} X_{es} = D_{s} \qquad \text{for all} \;\;s \in S$$

2. Overtime Constraints:

$$X_{4s} \leq M_{s} \qquad \text{for all} \;\;s \in S$$

3. Employee Type Constraints:

$$X_{7s}, X_{8s}, X_{9s} \leq U_{\text{Part-time}}$$

$$X_{10s}, X_{11s}, X_{12s} \leq U_{\text{Temporary}}$$

$$X_{13s}, X_{14s}, X_{15s} \leq U_{\text{On-call}}$$

$$X_{16s}, X_{17s}, X_{18s} \leq U_{\text{Vacation}}$$

$$X_{19s}, X_{20s} \leq U_{\text{Training}}$$

4. Non-negativity constraints:

$$ X_{es} \geq 0 \quad(\text{for all}\; e\in E,\; s\in S)$$ 


In [ ]:
# add variables

# Bibliography